In [1089]:
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [1090]:
load_dotenv()

raw_data_path = os.getenv('RAW_DATA_PATH')

# Checks to see that the CSV is properly loaded and ready to go into a dataframe

if raw_data_path is None:
    print("Error: CSV not found in .env file.")
else:
    print(f"CSV loaded successfully!")

CSV loaded successfully!


In [1091]:
df = pd.read_csv(raw_data_path)
print(df.head())
print(df.tail())

  Carrier Code Date (MM/DD/YYYY)  Flight Number Tail Number  \
0           AS        01/01/2023            2.0      N525AS   
1           AS        01/01/2023            4.0      N513AS   
2           AS        01/01/2023            8.0      N973AK   
3           AS        01/01/2023           12.0      N472AS   
4           AS        01/01/2023           16.0      N977AK   

  Destination Airport Scheduled departure time Actual departure time  \
0                 DCA                    14:05                 14:01   
1                 DCA                    08:20                 09:04   
2                 EWR                    07:40                 07:38   
3                 BOS                    07:35                 07:28   
4                 MCO                    07:55                 07:54   

   Scheduled elapsed time (Minutes)  Actual elapsed time (Minutes)  \
0                             292.0                          272.0   
1                             294.0             

In [1092]:
# Drop carrier code column and last 2 rows
df = df.drop(df.columns[0], axis=1)
df = df.iloc[:-2]

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78271 entries, 0 to 78270
Data columns (total 16 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Date (MM/DD/YYYY)                         78271 non-null  object 
 1   Flight Number                             78271 non-null  float64
 2   Tail Number                               78271 non-null  object 
 3   Destination Airport                       78271 non-null  object 
 4   Scheduled departure time                  78271 non-null  object 
 5   Actual departure time                     78271 non-null  object 
 6   Scheduled elapsed time (Minutes)          78271 non-null  float64
 7   Actual elapsed time (Minutes)             78271 non-null  float64
 8   Departure delay (Minutes)                 78271 non-null  float64
 9   Wheels-off time                           78271 non-null  object 
 10  Taxi-Out time (Minutes)           

In [1093]:
# Convert column names to snake case, remove text in parentheses, replace hyphens with underscores
df.columns = df.columns.str.replace(r'\s*\(.*?\)\s*', '', regex=True)
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.columns = df.columns.str.replace('-', '_')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78271 entries, 0 to 78270
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   date                            78271 non-null  object 
 1   flight_number                   78271 non-null  float64
 2   tail_number                     78271 non-null  object 
 3   destination_airport             78271 non-null  object 
 4   scheduled_departure_time        78271 non-null  object 
 5   actual_departure_time           78271 non-null  object 
 6   scheduled_elapsed_time          78271 non-null  float64
 7   actual_elapsed_time             78271 non-null  float64
 8   departure_delay                 78271 non-null  float64
 9   wheels_off_time                 78271 non-null  object 
 10  taxi_out_time                   78271 non-null  float64
 11  delay_carrier                   78271 non-null  float64
 12  delay_weather                   

In [1094]:
# Recast data types of columns

# Convert string and datetime columns to correct data type
df['tail_number'] = df['tail_number'].astype('string')
df['destination_airport'] = df['destination_airport'].astype('string')

# 'flight_number' needs to be converted into a string and back so we can remove the '.0' from every value
df['flight_number'] = df['flight_number'].astype('string')
df['flight_number'] = df['flight_number'].str.split('.').str[0]
df['flight_number'] = df['flight_number'].astype('string')

In [1095]:
def fix_time(time_str, date_str):
    # Handle nulls
    if pd.isna(time_str) or pd.isna(date_str):
        return np.nan
    
    # Clean strings
    time_str = str(time_str).strip()
    date_str = str(date_str).strip()
    
    try:
        base_date = datetime.strptime(date_str, '%m/%d/%Y')
        if time_str == '24:00':
            return base_date + timedelta(days=1)
        hours, minutes = map(int, time_str.split(':'))
        # Add a day for early morning times
        if 0 <= hours <= 3 and (hours < 3 or (hours == 3 and minutes <= 29)):
            base_date += timedelta(days=1)
        
        return base_date + timedelta(hours=hours, minutes=minutes)
    except (ValueError, TypeError):
        return np.nan

# Loop through time columns with fix_time()
df['date'] = df['date'].astype(str)
time_cols = ['scheduled_departure_time', 'actual_departure_time', 'wheels_off_time']

for col in time_cols:
    if col in df.columns:
        df[col] = df.apply(lambda row: fix_time(row[col], row['date']), axis=1)
        df[col] = pd.to_datetime(df[col])

print(df)

             date flight_number tail_number destination_airport  \
0      01/01/2023             2      N525AS                 DCA   
1      01/01/2023             4      N513AS                 DCA   
2      01/01/2023             8      N973AK                 EWR   
3      01/01/2023            12      N472AS                 BOS   
4      01/01/2023            16      N977AK                 MCO   
...           ...           ...         ...                 ...   
78266  12/31/2023          1314      N607AS                 SNA   
78267  12/31/2023          1316      N491AS                 LAX   
78268  12/31/2023          1318      N315AS                 SFO   
78269  12/31/2023          1326      N619AS                 SNA   
78270  12/31/2023          1328      N479AS                 OAK   

      scheduled_departure_time actual_departure_time  scheduled_elapsed_time  \
0          2023-01-01 14:05:00   2023-01-01 14:01:00                   292.0   
1          2023-01-01 08:20:00   20

In [1096]:
# Normalizing date in case we need to do analysis on date differences
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y').dt.normalize()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78271 entries, 0 to 78270
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            78271 non-null  datetime64[ns]
 1   flight_number                   78271 non-null  string        
 2   tail_number                     78271 non-null  string        
 3   destination_airport             78271 non-null  string        
 4   scheduled_departure_time        78271 non-null  datetime64[ns]
 5   actual_departure_time           78271 non-null  datetime64[ns]
 6   scheduled_elapsed_time          78271 non-null  float64       
 7   actual_elapsed_time             78271 non-null  float64       
 8   departure_delay                 78271 non-null  float64       
 9   wheels_off_time                 78271 non-null  datetime64[ns]
 10  taxi_out_time                   78271 non-null  float64       
 11  de

In [1097]:
# Identify columns currently listed as floats
float_columns = df.select_dtypes(include='float64').columns

# Function to check if a column can be converted to integers
def can_convert_to_int(column):
    return (column == column.astype('Int64')).all()

# List of columns that can be converted to integers
int_convertible_columns = [col for col in float_columns if can_convert_to_int(df[col])]

# Loop through and convert the columns to integers
for col in int_convertible_columns:
    df[col] = df[col].astype('Int64') # Int64 type will raise an error if NaN/None pops up

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78271 entries, 0 to 78270
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            78271 non-null  datetime64[ns]
 1   flight_number                   78271 non-null  string        
 2   tail_number                     78271 non-null  string        
 3   destination_airport             78271 non-null  string        
 4   scheduled_departure_time        78271 non-null  datetime64[ns]
 5   actual_departure_time           78271 non-null  datetime64[ns]
 6   scheduled_elapsed_time          78271 non-null  Int64         
 7   actual_elapsed_time             78271 non-null  Int64         
 8   departure_delay                 78271 non-null  Int64         
 9   wheels_off_time                 78271 non-null  datetime64[ns]
 10  taxi_out_time                   78271 non-null  Int64         
 11  de

## Data Cleaning

### All Columns
[x] Data range constraints\
[x] Uniqueness constraints\
[x] Missing data

### String Columns
[x] Length violations\
[x] Inconsistent formatting

### Datetime Columns
[x] Unit uniformity (handled in recasting)\
[ ] Crossfield validation

### Numeric Columns
[x] Unit uniformity (handled in recasting)\
[ ] Crossfield validation

In [1098]:
# Display any duplicate rows
df[df.duplicated(keep=False)]

,date,flight_number,tail_number,destination_airport,scheduled_departure_time,actual_departure_time,scheduled_elapsed_time,actual_elapsed_time,departure_delay,wheels_off_time,taxi_out_time,delay_carrier,delay_weather,delay_national_aviation_system,delay_security,delay_late_aircraft_arrival


In [1099]:
# Check for nulls and store the results in a DataFrame
null_counts_per_column = df.isnull().sum().reset_index()
null_counts_per_column.columns = ['Column Name', 'Null Count']

print(null_counts_per_column)

                       Column Name  Null Count
0                             date           0
1                    flight_number           0
2                      tail_number           0
3              destination_airport           0
4         scheduled_departure_time           0
5            actual_departure_time           0
6           scheduled_elapsed_time           0
7              actual_elapsed_time           0
8                  departure_delay           0
9                  wheels_off_time           0
10                   taxi_out_time           0
11                   delay_carrier           0
12                   delay_weather           0
13  delay_national_aviation_system           0
14                  delay_security           0
15     delay_late_aircraft_arrival           0


In [1100]:
# Data range constraints

# Check that all dates are in 2023

# Check that all time columns fall within logical range constraints

# Check that relevant int64 columns are non-negative
columns_to_check = ['scheduled_elapsed_time', 'actual_elapsed_time', 'taxi_out_time', 'delay_carrier', 'delay_weather', 'delay_national_aviation_system', 'delay_security', 'delay_late_aircraft_arrival']

summary_data = []

for column in columns_to_check:
    negative_count = (df[column] < 0).sum()
    summary_data.append({'Column': column, 'Negative_Count': negative_count})

summary_df = pd.DataFrame(summary_data)

print(summary_df)

                           Column  Negative_Count
0          scheduled_elapsed_time               0
1             actual_elapsed_time               0
2                   taxi_out_time               0
3                   delay_carrier               0
4                   delay_weather               0
5  delay_national_aviation_system               0
6                  delay_security               0
7     delay_late_aircraft_arrival               0


In [1101]:
# Length violations (strings)

# Aiport codes can't be more than 4 characters
columns_to_check = ['destination_airport']

results = []

for column in columns_to_check:
    count_exceeding = df[column].astype(str).apply(len).gt(4).sum()
    results.append((column, count_exceeding))

summary_df = pd.DataFrame(results, columns=['Column Name', 'Count > 4 Characters'])

print(summary_df)

# Flight numbers and tail numbers can't be greater than 6 characters
columns_to_check = ['flight_number', 'tail_number']

results = []

for column in columns_to_check:
    count_exceeding = df[column].astype(str).apply(len).gt(6).sum()
    results.append((column, count_exceeding))

summary_df = pd.DataFrame(results, columns=['Column Name', 'Count > 6 Characters'])

print(summary_df)

           Column Name  Count > 4 Characters
0  destination_airport                     0
     Column Name  Count > 6 Characters
0  flight_number                     0
1    tail_number                     0


In [1102]:
# Cross-validation

# Check that actual depature time - scheduled departure time = departure delay

# Check if calculated delays match stored delays and show validation summary
def validate_delays(df):
    # Calculate time differences
    calculated_delay = ((df['actual_departure_time'] - df['scheduled_departure_time'])
                       .dt.total_seconds() / 60
                       ).round().astype('int64')
    
    # Create validation dataframe
    validation_df = pd.DataFrame({
        'calculated_delay': calculated_delay,
        'stored_delay': df['departure_delay'],
        'delay_match': np.isclose(
            calculated_delay,
            df['departure_delay'],
            rtol=1e-10,
            atol=1.0,
            equal_nan=True
        )
    })
    
    # Find and summarize mismatches
    mismatches = validation_df[~validation_df['delay_match']]
    total_rows = len(validation_df)
    matching_rows = validation_df['delay_match'].sum()
    
    # Print summary
    print(f"Validation Summary:")
    print(f"Total rows: {total_rows}")
    print(f"Matching delays: {matching_rows} ({(matching_rows/total_rows*100):.2f}%)")
    print(f"Mismatched delays: {len(mismatches)} ({(len(mismatches)/total_rows*100):.2f}%)")
    
    # Show mismatch details if any exist
    if len(mismatches) > 0:
        print("\nFirst 10 mismatches:")
        mismatch_details = pd.concat([df, validation_df], axis=1).loc[mismatches.index]
        print(mismatch_details[['scheduled_departure_time', 'actual_departure_time', 
                               'calculated_delay', 'stored_delay', 'delay_match']].head(10))
    
    return validation_df

validate_delays(df)

# Original dataset reveals that the original departure_delay column was 0 for all actual departures at midnight, regardless of the real times

# Updating dataframe to include calculated delays
df['departure_delay'] = ((df['actual_departure_time'] - df['scheduled_departure_time'])
                        .dt.total_seconds() / 60
                        ).round().astype('int64')

# Verify the update
validate_delays(df)

Validation Summary:
Total rows: 78271
Matching delays: 77955 (99.60%)
Mismatched delays: 316 (0.40%)

First 10 mismatches:
    scheduled_departure_time actual_departure_time  calculated_delay  \
247      2023-01-02 23:10:00            2023-01-03                50   
371      2023-01-02 18:40:00            2023-01-03               320   
390      2023-01-02 06:00:00            2023-01-03              1080   
470      2023-01-03 14:20:00            2023-01-04               580   
479      2023-01-03 13:45:00            2023-01-04               615   
606      2023-01-04 07:55:00            2023-01-05               965   
652      2023-01-04 06:10:00            2023-01-05              1070   
657      2023-01-04 17:40:00            2023-01-05               380   
715      2023-01-04 15:35:00            2023-01-05               505   
795      2023-01-04 07:50:00            2023-01-05               970   

     stored_delay  delay_match  
247             0        False  
371             0 

,calculated_delay,stored_delay,delay_match
0,-4,-4,True
1,44,44,True
2,-2,-2,True
3,-7,-7,True
4,-1,-1,True
...,...,...,...
78266,-15,-15,True
78267,-6,-6,True
78268,-8,-8,True
78269,-8,-8,True


In [1103]:
# Add the delay time from all delay category columns to ensure they equal departure delay column

# Check that wheels-off time is always equal to or later than actual departure time

# Taxi-out time can't be greater than the difference between wheels-off time and actual departure timne

# The same tail number should not appear with a conflicting departure datetime

In [1104]:
# Formatting consistency (strings)

# Flight, tail, and destination airports can only have numbers or capital letters
columns_to_check = ['flight_number', 'tail_number', 'destination_airport']

summary_data = []

for column in columns_to_check:
    invalid_count = df[column].str.match(r'^[A-Z0-9]+$').sum()
    invalid_values_count = len(df) - invalid_count
    
    summary_data.append({'Column': column, 'Invalid_Count': invalid_values_count})

summary_df = pd.DataFrame(summary_data)

print(summary_df)

                Column  Invalid_Count
0        flight_number              0
1          tail_number              0
2  destination_airport              0


## Feature Engineering

In [1105]:
# Make an explicit index column 'trip_id' in case SQL/others need a primary key
df.insert(0, 'trip_id', range(1, len(df)+1)) # Starting at 1 for easy indexing in SQL/others
df.set_index('trip_id', inplace=True, drop=False)

print(df.info())
print(df)

<class 'pandas.core.frame.DataFrame'>
Index: 78271 entries, 1 to 78271
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   trip_id                         78271 non-null  int64         
 1   date                            78271 non-null  datetime64[ns]
 2   flight_number                   78271 non-null  string        
 3   tail_number                     78271 non-null  string        
 4   destination_airport             78271 non-null  string        
 5   scheduled_departure_time        78271 non-null  datetime64[ns]
 6   actual_departure_time           78271 non-null  datetime64[ns]
 7   scheduled_elapsed_time          78271 non-null  Int64         
 8   actual_elapsed_time             78271 non-null  Int64         
 9   departure_delay                 78271 non-null  int64         
 10  wheels_off_time                 78271 non-null  datetime64[ns]
 11  taxi_ou